In [1]:
import numpy as np
import pandas as pd
import datetime

import warnings
warnings.filterwarnings("ignore")

## Importing Analytical Report and calculating data

In [2]:
df = pd.read_excel('0FLATANALYTICALexport.XLSX')

In [3]:
#correction of Due date (-)
df['Due date (-)'] = df['Due date (-)'].astype('datetime64[ns]')
df['Due date (-)'][df['Due date (-)'] == '1816-03-29 05:56:08.066277376' ] = 0
df['test'] = np.where(df['Due date (-)'] == '1970-01-01', 0, 1)

# correction of Title transfer date (-)
df['Title transfer date (-)'] = df['Title transfer date (-)'].astype(object).where(df['Title transfer date (-)'].notnull(), 0)
df['Title transfer date (-)'] = np.where(df['Title transfer date (-)'] == 0, pd.to_datetime('today') + np.timedelta64(30,'D') , df['Title transfer date (-)'])
df['Title transfer date (-)'] = df['Title transfer date (-)'].astype('datetime64[ns]')

#continuation of correction of Due date (-)
df['Due date (-) corrected'] = np.where(df['test'] == 1, df['Due date (-)'], df['Due date (+)'] - df['Title transfer date (+)'] + df['Title transfer date (-)'])

df['Due date (-)'] = df['Due date (-) corrected']

# converting the type of Due date (-)
df['Due date (-)'] = df['Due date (-)'].astype('datetime64[ns]')
df['Due date (-)'] = df['Due date (-)'].dt.date

df['Due date (+)'] = df['Due date (+)'].dt.date
df['Title transfer date (-)'] = df['Title transfer date (-)'].dt.date
df['Title transfer date (+)'] = df['Title transfer date (+)'].dt.date


#filling NaN
df['Incoterms 1 (-)'].fillna(df['Incoterms 1 (+)'], inplace=True)
df['Incoterms 2 country (-)'].fillna(df['Incoterms 2 country (+).1'], inplace=True)
df['Incoterms 2 country (-).1'].fillna(df['Incoterms 2 country (+)'], inplace=True)


In [4]:
# calculating the columns

df['Consumption Date'] = df['Due date (+)']
df['Consumption Type'] = 'Filled by Macros'
df['Total WC Consumption period'] = df['Due date (-)'] - df['Due date (+)']
df['TTD difference'] = df['Title transfer date (-)'] - df['Title transfer date (+)']
df['Sale Payment Terms'] = df['Due date (-)'] - df['Title transfer date (-)']
df['Buy Payment Terms'] = df['Due date (+)'] - df['Title transfer date (+)']
df['PDD difference'] = df['Sale Payment Terms'] - df['Buy Payment Terms']
df['Consumption reason'] = 'OTHER'
df['WC Amount'] = df['WC consumption amount'] / 1000000
df['Current amount in USD (+)'] = df['Quantity BBL (ratio) (+)'] * df['Current price, USD/BBL (+)']
df['Current amount in USD (-)'] = df['Quantity BBL (ratio) (-)'] * df['Current price, USD/BBL (-)']

df['Company Name'] = df['Line of Business'].str[:3]

df['Portfolio name change'] = df['Company Name'] + ' ' + df['Portfolio name']
df['Portfolio name'] = df['Portfolio name change']


# converting columns to the right format
#converting into days
df['Total WC Consumption period'] = df['Total WC Consumption period'].dt.days
df['TTD difference'] = df['TTD difference'].dt.days
df['Sale Payment Terms'] = df['Sale Payment Terms'].dt.days
df['Buy Payment Terms'] = df['Buy Payment Terms'].dt.days
df['PDD difference'] = df['PDD difference'].dt.days

#fixing the counterparty where Litasco appears
df['Counterparty (+)'] = np.where(df['Counterparty (+)'] == 'LITASCO SA', df['Company Name'] + ' ' + df['Mirror portfolio name (+)'], 
                                  df['Counterparty (+)'])

df['Counterparty (-)'].fillna('', inplace=True)
df['Counterparty (-)'] = np.where(df['Counterparty (-)'] == 'LITASCO SA', df['Company Name'] + ' ' + df['Mirror portfolio name (-)'], 
                                  df['Counterparty (-)'])


In [5]:
# correcting the WC Amount
df_strategy = pd.DataFrame()
df_strategy = df[['Strategy', 'WC Amount','Strategy start date']].copy()


#import the strategies
df_strategy_exclude = pd.read_excel('Strategies.xlsx')

#conditions and sum
df_strategy = df_strategy.merge(df_strategy_exclude, on = 'Strategy', how = 'left')
df_strategy['New'].fillna(0, inplace=True)

df_strategy['S2'] = np.where(df_strategy['Strategy'] > 517000, 1,0)
df_strategy['ssd'] = np.where(df_strategy['Strategy start date'] > '2015-12-01', 1,0)

df_strategy['sum_s'] = df_strategy['New'] + df_strategy['ssd'] + df_strategy['S2']

df_strategy['WC Amount corrected'] = np.where(df_strategy['sum_s'] > 0,df_strategy['WC Amount'], 0)

# replacing WC amount
df['WC Amount'] = df_strategy['WC Amount corrected']
# rounding WC amount
df['WC Amount'] = np.round(df['WC Amount'], decimals=1)

FileNotFoundError: [Errno 2] No such file or directory: 'Strategies.xlsx'

In [5]:
def f(row):
    if (row['Total WC Consumption period'] >= row['TTD difference'] and row['Total WC Consumption period'] > 0 and row['TTD difference'] > 0) \
    or (row['Total WC Consumption period'] <= row['TTD difference'] and row['Total WC Consumption period'] < 0 and row['TTD difference'] < 0):
        return row['TTD difference']
    elif (row['Total WC Consumption period'] > 0 and row['TTD difference'] > 0) \
    or (row['Total WC Consumption period'] < 0 and row['TTD difference'] < 0):
        return row['Total WC Consumption period']
    else:
        return 0

In [6]:
df['TTD influence on WC consumption'] = df.apply(f, axis=1)
df['PDD influence on WC consumption'] = df['Total WC Consumption period'] - df['TTD influence on WC consumption']

In [7]:
# rearrange columns
cols = df.columns.tolist()

In [8]:
cols = ['Consumption Date',
       'Consumption Type',
       'Total WC Consumption period',
        'TTD influence on WC consumption',
        'PDD influence on WC consumption',
        'TTD difference',
        'PDD difference',
        'Sale Payment Terms',
        'Buy Payment Terms',
        'Consumption reason',
        'WC Amount',
        'Buy/Sell (-)',
        'Buy/Sell (+)',
        'Due date (-)',
        'Due date (+)',
        'Title transfer date (-)',
        'Title transfer date (+)',
        'Strategy',
        'Strategy name',
        'Company Name',
        'Global Book name',
        'Line of Business',
        'Book Name',
        'Portfolio name',
        'Quantity',
        'Current amount in USD (+)',
        'Quantity BBL (ratio) (+)',
        'Current price, USD/BBL (+)',
        'Counterparty (+)',
        'Title transfer (+)',
        'Incoterms 1 (+)',
        'Incoterms 2 country (+)',
        'Incoterms 2 country (+).1',
        'Object type (+)',
        'Delivery ID (+)',
        'Internal type (+)',
        'Mirror Delivery ID (+)',
        'Current amount in USD (-)',
        'Quantity BBL (ratio) (-)',
        'Current price, USD/BBL (-)',
        'Counterparty (-)',
        'Title transfer (-)',
        'Incoterms 1 (-)',
        'Incoterms 2 country (-)',
        'Incoterms 2 country (-).1',
        'Object type (-)',
        'Delivery ID (-)',
        'Internal type (-)',
        'Mirror Delivery ID (-)',
        'Strategy start date',
        'Strategy end date',
        'Voyage ID (+)',
        'Transport name (+)',
        'Grade (+)',
        'Voyage ID (-)',
        'Transport name (-)',
        'Grade (-)']

In [9]:
# ordering the columns
df = df[cols]
df.shape

(9662, 57)

In [10]:
# removing rows for which WC Amount is zero, 
df = df[df['WC Amount'] != 0]
df.shape

(9662, 57)

In [11]:
# selecting rows for which dates are above -365 and below 365
df = df[(df['Total WC Consumption period'] > -365) & (df['Total WC Consumption period'] < 365) ]
df.shape

(9454, 57)

In [12]:
# reset index
df = df.reset_index(drop=True)
#df = df.drop('index', 1)

In [13]:
# formatting the dates to day-month-year
df['Consumption Date'] = pd.to_datetime(df['Consumption Date'])
df['Consumption Date'] = df['Consumption Date'].dt.strftime('%d/%m/%Y')

df['Due date (+)'] = pd.to_datetime(df['Due date (+)'])
df['Due date (+)'] = df['Due date (+)'].dt.strftime('%d/%m/%Y')


df['Due date (-)'] = pd.to_datetime(df['Due date (-)'])
df['Due date (-)'] = df['Due date (-)'].dt.strftime('%d/%m/%Y')

df['Title transfer date (+)'] = pd.to_datetime(df['Title transfer date (+)'])
df['Title transfer date (+)'] = df['Title transfer date (+)'].dt.strftime('%d/%m/%Y')

df['Title transfer date (-)'] = pd.to_datetime(df['Title transfer date (-)'])
df['Title transfer date (-)'] = df['Title transfer date (-)'].dt.strftime('%d/%m/%Y')

#df['Strategy start date'] = pd.to_datetime(df['Strategy start date'])
#df['Strategy start date'] = df['Strategy start date'].dt.strftime('%d/%m/%Y')

#df['Strategy end date'] = pd.to_datetime(df['Strategy end date'])
#df['Strategy end date'] = df['Strategy end date'].dt.strftime('%d/%m/%Y')

#df['Strategy end date']


In [14]:
# removing the rows for which will be below the minimum date even adding the Total WC period
df['Consumption Date'] = pd.to_datetime(df['Consumption Date'])
#df['Date Min'] = pd.to_datetime('2016-12-31')
#df['Date CTRL 1'] = pd.to_timedelta(abs(df['Total WC Consumption period']), unit ='D')
#df['Date CTRL 2'] = df['Consumption Date'] + df['Date CTRL 1']

# filter data
#df = df[(df['Date CTRL 2'] > '2016-12-31')]

# reset index
df = df.reset_index(drop=True)
df.shape

(9454, 57)

In [15]:
# ordering the columns (removing the three added cols)
df = df[cols]
df.shape

(9454, 57)

## Exporting the file

In [ ]:

# export to excel
writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
# ****************** columns to correct***************************
# counterparty (+), some differences in method. not sure if it adds value changing...
# counterparty (-), some differences in method. not sure if it adds value changing...


In [24]:
#import os
#file = "C:\Users\DAPE\Desktop\Sandbox\Python\Tableau_data.xlsm"
#os.startfile(file)
df.shape

(9454, 57)

## Stock in Transit and Payment Terms

In [21]:
# stock in transit
stock_in_transit = df.loc[df.index.repeat(abs(df['TTD influence on WC consumption']))]
stock_in_transit.loc[:,'Consumption Date'] += pd.to_timedelta(stock_in_transit.groupby(level=0).cumcount(),'D')
stock_in_transit = stock_in_transit.reset_index(drop=True)
stock_in_transit['Consumption Type'] = 'Stock in Transit'

In [23]:
# payment terms
payment_terms = df.loc[df.index.repeat(abs(df['PDD influence on WC consumption']))]
payment_terms.loc[:,'Consumption Date'] += pd.to_timedelta(payment_terms.groupby(level=0).cumcount(),'D')
payment_terms = payment_terms.reset_index(drop=True)
payment_terms['Consumption Type'] = 'Payment Terms'

In [25]:
stock_in_transit.shape

(142155, 57)

In [26]:
payment_terms.shape

(75835, 57)

In [28]:
df_new = pd.concat([stock_in_transit, payment_terms])
df_new.shape

(217990, 57)

In [31]:
df_new = df_new.reset_index(drop=True)

In [32]:
df_new

,Consumption Date,Consumption Type,Total WC Consumption period,TTD influence on WC consumption,PDD influence on WC consumption,TTD difference,PDD difference,Sale Payment Terms,Buy Payment Terms,Consumption reason,...,Internal type (-),Mirror Delivery ID (-),Strategy start date,Strategy end date,Voyage ID (+),Transport name (+),Grade (+),Voyage ID (-),Transport name (-),Grade (-)
0,2015-02-01,Stock in Transit,5,5,0,25,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05871,ASTRO POLARIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)
1,2015-02-02,Stock in Transit,5,5,0,25,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05871,ASTRO POLARIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)
2,2015-02-03,Stock in Transit,5,5,0,25,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05871,ASTRO POLARIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)
3,2015-02-04,Stock in Transit,5,5,0,25,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05871,ASTRO POLARIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)
4,2015-02-05,Stock in Transit,5,5,0,25,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05871,ASTRO POLARIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)
5,2015-02-01,Stock in Transit,11,11,0,31,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05995,IOANNIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)
6,2015-02-02,Stock in Transit,11,11,0,31,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05995,IOANNIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)
7,2015-02-03,Stock in Transit,11,11,0,31,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05995,IOANNIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)
8,2015-02-04,Stock in Transit,11,11,0,31,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05995,IOANNIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)
9,2015-02-05,Stock in Transit,11,11,0,31,-20,15,35,OTHER,...,NaN,NaN,2000-01-01,9999-12-31 00:00:00,VGC14_02868,ASTRO POLARIS,URALS (EXPORT BLEND CRUDE OIL),VGC14_05995,IOANNIS,URALS (RUSSIAN EXPORT BLEND CRUDE OIL)


## Next steps
1. Actuals
2. Calculate Others
3. Fill Others info
4. Adjustments